It's easiest to implement the S-Box as a lookup table

In [165]:
sbox_lookup = [
0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16,
]

def int_to_bin(number:int, mode="AES"):
    if mode == "AES":
        return '{0:08b}'.format(number)
    elif mode == "DES_in":
        return '{0:06b}'.format(number)
    else: 
        return '{0:04b}'.format(number)

def get_hamming_weight(number:int):
    return sum([number&(1<<i)>0 for i in range(8)])

def dot_product(first:int, second:int):
    product = first&second
    hw = get_hamming_weight(product)
    return hw % 2

#The AES S-box gets an integer and returns the corresponding integer
def AES_sbox(message:int):
    return sbox_lookup[message]

In [166]:
def palphabetaS_AES(alpha:int, beta:int):
    occurances = {0:0,1:0}
    for x in range(256):
        oplus_sum = (dot_product(alpha, x) + dot_product(beta, AES_sbox(x)))%2
        occurances[oplus_sum] += 1
    return occurances

In [167]:
def find_best_relation_AES():
    max_alpha = 0
    max_beta = 0
    max_value = 0
    for alpha in range(256):
        if (alpha%50) == 0:
            print(alpha)
        for beta in range(256):
            if alpha==0 and beta==0:
                continue
            occurances = palphabetaS_AES(alpha,beta)
            better_probability_set = max(occurances, key = occurances.get)
            better_probability = occurances[better_probability_set]
            if better_probability > max_value:
                max_value = better_probability
                max_alpha = alpha
                max_beta = beta
    return max_alpha,max_beta

In [168]:
find_best_relation_AES()

0
50
100
150
200
250


(1, 72)

In [153]:
palphabetaS_AES(1,72)

{0: 112, 1: 144}

In [155]:
potential = (2*(144/256)-1)**2
potential

0.015625

Now we want to do the same for DES

In [193]:
des_sbox_lookup = [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
[14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
[4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
[11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]]

def DES_sbox(message:int):
    binary = int_to_bin(message, "DES_in")
    row = binary[0]+binary[5]
    column = binary[1]+binary[2]+binary[3]+binary[4]
    return des_sbox_lookup[int(row, 2)][int(column, 2)]

def palphabetaS_DES(alpha:int, beta:int):
    occurances = {0:0,1:0}
    for x in range(2**6):
        oplus_sum = (dot_product(alpha, x) + dot_product(beta, DES_sbox(x)))%2
        occurances[oplus_sum] += 1
    return occurances

In [195]:
def find_best_relation_DES():
    max_alpha = 0
    max_beta = 0
    max_value = 0
    for alpha in range(2**6):
        if (alpha%50) == 0:
            print(alpha)
        for beta in range(2**4):
            if alpha==0 and beta==0:
                continue
            occurances = palphabetaS_DES(alpha,beta)
            better_probability_set = max(occurances, key = occurances.get)
            better_probability = occurances[better_probability_set]
            if better_probability > max_value:
                max_value = better_probability
                max_alpha = alpha
                max_beta = beta
    return max_alpha,max_beta

In [196]:
find_best_relation_DES()

0
50


(16, 15)

In [200]:
palphabetaS_DES(16,15)

{0: 12, 1: 52}

In [201]:
potential = (2*(52/64)-1)**2
potential

0.390625